In [1]:
"如果不担心数据覆盖可以直接运行到底"
import networkx as nx
import os
from tqdm import  tqdm
import pandas as pd

output_folder=r"result\YearStat"
data_folder=r"result\YearOutput"

In [4]:

node_set=set()
for file in tqdm(os.listdir(data_folder)):
    year,_=file.split(".")
    year_G=nx.read_graphml(os.path.join(data_folder, file))
    node_set.update(year_G.nodes)

100%|██████████| 13/13 [00:04<00:00,  2.68it/s]


In [7]:
year_G.nodes['MKS仪器公司']

{'country': '美国', 'type': 'Company', 'name': 'MKS仪器公司'}

In [ ]:
node_list=list(node_set)
data_list_of_dict={node: {} for node in node_list}  # 后续每一个节点的数据都存在这里
len(node_list)

60384

In [23]:
def count_stopped_edges(graph: nx.DiGraph, as_target=True) -> dict:
    """
    计算NetworkX DiGraph中每个节点在status="Stopped"的连边中作为target的次数。
    
    参数:
        graph (nx.DiGraph): 输入的有向图，假设边有一个属性'status'。
        as_target: 用来实现两个不同功能，默认True统计被断供的企业，False表示统计断供企业
        
    返回:
        dict: 一个字典，键是图中的节点，值是在状态为"Stopped"的边中作为target的次数。
    """
    # 初始化结果字典
    count_stopped = {node: 0 for node in graph.nodes()}
    
    # 遍历图中的所有边
    for u, v, data in graph.edges(data=True):
        # 检查边是否具有'status'属性且值为'Stopped'
        if 'status' in data and data['status'] == "Stopped":
            # 增加作为target的节点v的计数
            if as_target:
                count_stopped[v] += 1  # 统计被断供者
            else:
                count_stopped[u] += 1  # 统计断供者
            
    # 过滤掉计数为0的项
    # count_stopped = {k: v for k, v in count_stopped.items() if v > 0}
    
    return count_stopped

In [24]:
# 每次读取一个文件，这个文件中每一个公司的信息当做一行来保存

all_data_list=[]

for file in tqdm(os.listdir(data_folder)):
    year, _ = file.split(".")
    year_G = nx.read_graphml(os.path.join(data_folder, file))
    stopped_count = count_stopped_edges(year_G, as_target=True)
    stopper_count = count_stopped_edges(year_G, as_target=False)
    
    edges_to_remove = [edge for edge in year_G.edges 
                       if year_G.edges[edge].get("status", "") == "Stopped"]
    
    # 计算节点度数并存储
    year_degree = dict(year_G.degree())
    year_degree_centrality = nx.degree_centrality(year_G)
    year_closeness_centrality = nx.closeness_centrality(year_G)
    year_between_centrality = nx.betweenness_centrality(year_G)
    year_pagerank = nx.pagerank(year_G)
    
    data_map_dict={
        "stopped_count": stopped_count,
        "stopper_count": stopper_count,
        "degree": year_degree,
        "degree_centrality": year_degree_centrality,
        "closeness_centrality": year_closeness_centrality,
        "betweenness_centrality": year_between_centrality,
        "pagerank": year_pagerank
    }
    
    for node in tqdm(year_G.nodes):
        record={"year": year, "name": node, **year_G.nodes[node]}
        for key, value_map in data_map_dict.items():
            record[key]=value_map[node]
        
        all_data_list.append(record)
    # break

100%|██████████| 13/13 [09:05<00:00, 41.93s/it]


In [25]:
pd.DataFrame(all_data_list).to_csv(r"result\YearStat\timeSeries_with_stopped.csv", index_label="id")